This model tries to optimize stress-strain state (reduce displacements to acceptable level) by combination of simple finite element analysis and reinforcement learning

# Finite Element Model of Axially Loaded Bar

In [1]:
# Details of model can be found at:
# https://en.wikiversity.org/wiki/Introduction_to_finite_elements/Axial_bar_finite_element_solution

In [2]:
import numpy as np

In [3]:
def elementStiffness(A, E, h):
    s= A*E/h
    return s*np.array([[1,-1],[-1,1]])

In [4]:
def elementLoad(node1, node2, a, h):

    x1 = node1
    x2 = node2

    fe1 = a*x2/(2*h)*(x2**2-x1**2) - a/(3*h)*(x2**3-x1**3)
    fe2 = -a*x1/(2*h)*(x2**2-x1**2) + a/(3*h)*(x2**3-x1**3)
    return np.array([fe1,fe2])

In [5]:
def AxialBarFEM(A,E):
    L = 1.0
    a = 1.0
    R = 1.0    
    e = 3
    h = L/e
    n = e+1
        
    node=[]    
    for i in range(n):
        node.append(i*h)
    node=np.array(node) 
        
    elem=[]    
    for i in range(e):
        P=[i,i+1]
        elem.append(P)
    elem=np.array(elem)    
      
    K=np.zeros((n,n))   
    f=np.zeros((n,1))  
       
    for i in range(e):
        node1 = elem[i,0]
        node2 = elem[i,1]
        Ke = elementStiffness(A, E, h)
        fe = elementLoad(node[node1],node[node2], a, h)
        K[node1:node2+1,node1:node2+1] = K[node1:node2+1,node1:node2+1] + Ke
        f[node1:node2+1] = f[node1:node2+1] + fe.reshape(2,1)
         
    f[n-1] = f[n-1] + 1.0
   
    Kred = K[1:n,1:n]
    fred = f[1:n] 
    d = np.dot(np.linalg.inv(Kred),fred)  
    dsol = np.insert(d,0,0)   
                
    return  dsol[-1],A,E

In [6]:
# Input: cross-sectional area and Young's modulus
# Output: largest displacement at rightmost node at the point of external force application

AxialBarFEM(0.5, 1)

(2.6666666666666665, 0.5, 1)

# Neural Network Policy - Policy Gradients

In [7]:
# Details of model can be found in the book:
# Hands-On Machine Learning with Scikit-Learn & TensorFlow

In [8]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
n_inputs = 3 
n_hidden = 50 
n_outputs = 9 
initializer = tf.contrib.layers.variance_scaling_initializer()

learning_rate = 0.001

# Build the neural network
X_ = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X_")
hidden = fully_connected(X_, n_hidden, activation_fn=tf.nn.elu, weights_initializer=initializer)
hidden1 = fully_connected(hidden, n_hidden, activation_fn=tf.nn.elu, weights_initializer=initializer)
logits = fully_connected(hidden1, n_outputs, activation_fn=None, weights_initializer=initializer)
outputs = tf.nn.softmax(logits, name="Y_proba")

# Select a random action based on the estimated probabilities
action = tf.multinomial(tf.log(outputs), num_samples=1,output_dtype=tf.int32)

y=tf.reshape(tf.one_hot(action,depth=9,dtype=tf.float32),[9,1])
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=tf.transpose(logits))

optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(xentropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
    
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
def discount_rewards(rewards, discount_rate=0.97):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

In [11]:
def discount_and_normalize_rewards(all_rewards, discount_rate=0.97):
    all_discounted_rewards = [discount_rewards(rewards) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [12]:
def prestep(action,A,E):
    d=0.01
    d1=0.005
    if action==0:
        return A-d, E
    elif action==1:
        return A-d,E-d1
    elif action==2:
        return A-d,E+d1
    elif action==3:
        return A,E
    elif action==4:
        return A+d,E
    elif action==5:
        return A+d, E+d1
    elif action==6:
        return A+d, E-d1
    elif action==7:
        return A, E+d1
    else:
        return A,E-d1

In [13]:
def reward_(obs_,obs):
#     if obs_[1]>obs[1]: # use when minimizing cross-sectional area
 
    if obs_[0]>obs[0]:  # use when minimizing displacement  
        return 1
    else:
        return 0

In [14]:
import random

In [15]:
# training

In [16]:
n_iterations =100 #250 # number of training iterations
n_max_steps = 300 #1000 # max steps per episode
n_games_per_update = 10 # train the policy every 10 episodes
save_iterations = 10 # save the model every 10 training iterations

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = [] # all sequences of raw rewards for each episode
        all_gradients = [] # gradients saved at each step of each episode
              
        for game in range(n_games_per_update):
            current_rewards = [] # all raw rewards from the current episode
            current_gradients = [] # all gradients from the current episode
            
            A=3*random.random()
            E=2*random.random()
            obs=AxialBarFEM(A,E)            
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients],
                                                     feed_dict={X_:  np.array(obs).reshape(1,n_inputs)}) # one obs
                obs_=obs
                A,E=prestep(action_val[0][0],A,E)
                obs=AxialBarFEM(A,E)
                reward=reward_(obs_,obs)
                current_rewards.append(reward)
                current_gradients.append(gradients_val)

                if obs[0]<0.1: 
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        
        # At this point we have run the policy for 10 episodes, and we are
        # ready for a policy update using the algorithm described earlier.
        all_rewards = discount_and_normalize_rewards(all_rewards)
        
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            # multiply the gradients by the action scores, and compute the mean
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index] 
                                      for game_index, rewards in enumerate(all_rewards)
                                      for step, reward in enumerate(rewards)],axis=0)
            feed_dict[grad_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "./policy/my_policy_net_pg.ckpt")

In [17]:
# for op in tf.get_default_graph().get_operations():
#     print (str(op.name)) 

In [18]:
# prediction

In [89]:
def predict(thr,A,E):
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph('./policy/my_policy_net_pg.ckpt.meta')
        saver.restore(sess, "./policy/my_policy_net_pg.ckpt") 

        graph = tf.get_default_graph()
        outputs = graph.get_tensor_by_name("Y_proba:0") 
        X_ = graph.get_tensor_by_name("X_:0") 
        
        obs=AxialBarFEM(A,E)

        for step in range(100):
            action_val= sess.run([outputs],feed_dict={X_:  np.array(obs).reshape(1,n_inputs)}) 
            A,E=prestep(np.argmax(action_val),A,E)  
            obs=AxialBarFEM(A,E)
            if obs[0]<thr and obs[0]>0.8*thr: # use when minimizing displacement
                break
#             if obs[0]<thr or obs[0]>3*thr : # use when minimizing cross-sectional area
#                 break    
        if obs[0]>thr:
            return "Bad initial parameters! Try increasing initial cross-sectional area A, Young's modulus E and/or number of iterations"
        if obs[0]<0.8*thr:
            return "You can get better parameters. Try decreasing initial area A and/or Young's modulus E"
    return "Solution converged! MaxDispl={}, A={},E={}".format(obs[0],obs[1],obs[2])

In [90]:
# test

In [96]:
predict(thr=2,A=1.3,E=0.1)

INFO:tensorflow:Restoring parameters from ./policy/my_policy_net_pg.ckpt


'Solution converged! MaxDispl=1.9960079840319345, A=1.6700000000000004,E=0.40000000000000024'